# 1. Import the necessary libraries for you to be able to query 4square API

In [195]:
import requests
import json
import pandas as pd
from getpass import getpass
import cartoframes
from dotenv import load_dotenv
import os
import pymongo
token = os.getenv("token")

# 2. Get names and location of the 10 coffee places by passing Ironhack as your location

In [196]:
where = "Pamplona, 96, 08018 Barcelona"
url_geocode = f"https://geocode.xyz/{where}?json=1"
res = requests.get(url_geocode)
res.json()


{'standard': {'stnumber': '96',
  'addresst': 'Pamplona',
  'statename': 'Catalonia',
  'postal': '08018',
  'region': 'Catalonia',
  'prov': 'ES',
  'city': 'Barcelona',
  'countryname': 'Spain',
  'confidence': '0.9'},
 'longt': '2.19035',
 'alt': {},
 'elevation': {},
 'latt': '41.39780'}

In [197]:
lat = res.json()["latt"]
lon = res.json()["longt"]

In [198]:
print(lat, lon)

41.39780 2.19035


In [199]:
def response_foursquare (query, lat, lon, radius=500, limit=1): 
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }

    response = requests.get(url, headers=headers)
    return response.json()

In [200]:
def extracting_from_one_element (element):

    name = element["name"]
    lat = element["geocodes"]["main"]["latitude"]
    lon = element["geocodes"]["main"]["longitude"]

    
    small_dict = {
    "name": name,
    "lat": lat,
    "lon": lon
}
    return small_dict

In [201]:
response = response_foursquare("cafe", lat, lon, 500, 10)

In [202]:
coffee_shop_names = [extracting_from_one_element (element) for element in response["results"]]
coffee_shop_names

[{'name': 'Cafeteria Industrial', 'lat': 41.399175, 'lon': 2.195368},
 {'name': "L'Estoneta", 'lat': 41.399145, 'lon': 2.188496},
 {'name': 'Espai Joliu', 'lat': 41.398735, 'lon': 2.195069},
 {'name': 'Cafeteria Botanica', 'lat': 41.398044, 'lon': 2.191915},
 {'name': 'SKYE Coffee Co', 'lat': 41.396937, 'lon': 2.191519},
 {'name': 'Classic Bar Cafeteria', 'lat': 41.395408, 'lon': 2.187203},
 {'name': 'Cafeteria Bar BAU', 'lat': 41.397524, 'lon': 2.196263},
 {'name': "Nømad Roaster's Home", 'lat': 41.397977, 'lon': 2.195589},
 {'name': 'Rebeka', 'lat': 41.396763, 'lon': 2.18902},
 {'name': 'Syra Coffee Poblenou', 'lat': 41.396518, 'lon': 2.19417}]

# 3. Now get the 5 CLOSEST coffe places. Make use of params: limit, near?

In [203]:
url_2 = f"https://api.foursquare.com/v3/places/search?limit=5&near=Pamplona%2C%2096%2C%2008018%20Barcelona"

headers = {
    "accept": "application/json",
    "Authorization": token
}

response_2 = requests.get(url_2, headers=headers)
response_2 = response_2.json()
cofee_shop_names_2 = [extracting_from_one_element (element) for element in response_2["results"]]
cofee_shop_names_2

[{'name': 'Tandem Cocktail Bar', 'lat': 41.389615, 'lon': 2.157902},
 {'name': 'Roast Club', 'lat': 41.387825, 'lon': 2.157221},
 {'name': 'Garage Beer Co', 'lat': 41.387059, 'lon': 2.160146},
 {'name': 'DelaCrem', 'lat': 41.387965, 'lon': 2.161961},
 {'name': 'Hq Barcelona', 'lat': 41.390995, 'lon': 2.157773}]

# 4. Define a function that returns name, lat, long for starbucks OR restaurants AND for any given location

In [204]:
import requests

def foursquare_places(sb_or_restaurant, lat, lon):
    if sb_or_restaurant.lower() == 'starbucks':
        query = 'starbucks'
    elif sb_or_restaurant.lower() == 'restaurant':
        query = 'restaurant'
    else:
        return "Invalid venue type. Supported types: 'starbucks' or 'restaurant'."

    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}"

    headers = {
    "accept": "application/json",
    "Authorization": token
    }

    response = requests.get(url, headers=headers)
    data = response.json()
        
    get_names = [extracting_from_one_element (element) for element in data["results"]]

    return get_names


place = 'restaurant'
result = foursquare_places(place, lat, lon)
result


[{'name': 'Can Dendê', 'lat': 41.39836, 'lon': 2.198345},
 {'name': 'La Tavernícola', 'lat': 41.400358, 'lon': 2.197944},
 {'name': 'Little Fern', 'lat': 41.402328, 'lon': 2.197571},
 {'name': 'Parking Pizza', 'lat': 41.396301, 'lon': 2.174171},
 {'name': 'Casa Lolea', 'lat': 41.388833, 'lon': 2.177238},
 {'name': 'Funky Bakers', 'lat': 41.394916, 'lon': 2.172866},
 {'name': 'Forn Bertran', 'lat': 41.383552, 'lon': 2.194749},
 {'name': 'Bodega la Tinaja', 'lat': 41.384329, 'lon': 2.18309},
 {'name': 'El Xampanyet', 'lat': 41.384569, 'lon': 2.18178},
 {'name': 'Koy Shunka', 'lat': 41.385841, 'lon': 2.175345}]

# 5. Export it into data folder as a json: `data/coffe_shops.json`

In [205]:
with open('./data/coffee_shops', "w") as json_file:
    json.dump(coffee_shop_names, json_file, indent=4)

# BONUS: load it into MongoDB in a collection named "ironcoffee" using pymongo

In [206]:
import pymongo
client = pymongo.MongoClient("localhost:27017")
db = client["ironhack"]
c = db.get_collection("ironcofee") 

for shop in coffee_shop_names:
    c.insert_one(shop)